In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['items.csv', 'sample_submission.csv', 'test.csv', 'sales_train.csv', 'item_categories.csv', 'shops.csv']


In [2]:
import pandas as pd
import numpy as np
from itertools import product

In [3]:
sales = pd.read_csv('../input/sales_train.csv')
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [4]:
items = pd.read_csv('../input/items.csv')
test = pd.read_csv('../input/test.csv')
item_categories = pd.read_csv('../input/item_categories.csv')
shops = pd.read_csv('../input/shops.csv')

sales['date'] = pd.to_datetime(sales['date'], format='%d.%m.%Y')

prediction are need to be made based on monthly data

In [ ]:
index_cols = ['date_block_num', 'shop_id', 'item_id']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[[block_num], cur_shops, cur_items])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#####################
#get aggregated values for (shop_id, item_id, month)
#gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target': 'sum'}})
#fix column names
#gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]

# alternative way to sum item_cnt_day, without warning from pandas for using dict in agg()
gb = sales.groupby(index_cols,as_index=False).item_cnt_day.sum()
gb = gb.rename(columns={'item_cnt_day': 'item_cnt_month'})
#####################

#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)

#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

In [33]:
all_data.to_csv('all_data.csv', index=False)

In [34]:
pd.read_csv('all_data.csv')

,date_block_num,shop_id,item_id,item_cnt_month
0,0,59,22154,1.0
1,0,59,2552,0.0
2,0,59,2554,0.0
3,0,59,2555,0.0
4,0,59,2564,0.0
5,0,59,2565,0.0
6,0,59,2572,0.0
7,0,59,2573,0.0
8,0,59,2574,2.0
9,0,59,2593,0.0
